In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, KBinsDiscretizer
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
from scipy.sparse import hstack
from collections import defaultdict, Counter
from ds_tools.ds_tools import CategoricalTransformer
import matplotlib.pyplot as plt
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')
df_train.drop('id', axis=1, inplace=True)

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
test_surface = df_test['tc_mseki']
df_test.drop('id', axis=1, inplace=True)

## Preprocessing

In [3]:
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

def split_address(df):
    df['jukyo'] = df['jukyo'].str.slice(start=3).str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)
    return df

df_train = split_address(df_train)
df_test = split_address(df_test)

df_train.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

In [4]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))


combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

In [5]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
price_stats = []
for train_idx, valid_idx in splitter.split(df_train):
    price_stats_by_city = defaultdict(dict)
    for city, group in df_train.iloc[train_idx].groupby('city'):
        price_list = group['keiyaku_pr']/group['tc_mseki']
        price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
        price_stats_by_city[city]['price_by_city_median'] = price_list.median()
        price_stats_by_city[city]['price_by_city_min'] = price_list.min()
        price_stats_by_city[city]['price_by_city_max'] = price_list.max()
        price_stats_by_city[city]['price_by_city_std'] = price_list.std()
        price_stats_by_city[city]['price_by_city_count'] = len(price_list)
    for i, city in enumerate(df_train.iloc[valid_idx]['city']):
        price_stats.append((valid_idx[i], price_stats_by_city[city]))

price_stats_test = [] 
price_stats_by_city = defaultdict(dict)
for city, group in df_train.groupby('city'):
    price_list = group['keiyaku_pr']/group['tc_mseki']
    price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
    price_stats_by_city[city]['price_by_city_median'] = price_list.median()
    price_stats_by_city[city]['price_by_city_min'] = price_list.min()
    price_stats_by_city[city]['price_by_city_max'] = price_list.max()
    price_stats_by_city[city]['price_by_city_std'] = price_list.std()
    price_stats_by_city[city]['price_by_city_count'] = len(price_list)
for city in df_test['city']:
    price_stats_test.append(price_stats_by_city[city])
    
df_price_stats = pd.DataFrame([x[1] for x in sorted(price_stats, key=lambda x: x[0])])
df_price_stats_test = pd.DataFrame(price_stats_test)

df_train = pd.concat([df_train, df_price_stats], axis=1)
df_test = pd.concat([df_test, df_price_stats_test], axis=1)


for col in ['mseki_rd_hb', 'road3_fi', 'rosenka_hb', 'kempei2', 'road2_mg', 'kaoku_hb', 'bus_hon']:
    df_train[col].replace(0.0, np.nan, inplace=True)
    df_test[col].replace(0.0, np.nan, inplace=True)

In [6]:
df_train['station_name_prefix'] = df_train['rosen_nm1'].str.slice(stop=2)
df_test['station_name_prefix'] = df_test['rosen_nm1'].str.slice(stop=2)

## Train

In [7]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
continue_features += ['price_by_city_mean', 'price_by_city_median', 'price_by_city_min', 'price_by_city_max', 
                      'price_by_city_std', 'price_by_city_count']
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)
        
for col in categorical_features:
    if col not in ['pj_no']:
        ct = CategoricalTransformer(min_freq=3)
        df_train[col] = ct.fit_transform(df_train[col])
        df_test[col] = ct.transform(df_test[col])
        
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

## LGB

In [33]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']-1e6+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']-1e6+1)
    regressor = lgb.LGBMRegressor(n_estimators=100000, learning_rate=0.01, silent=False, random_state=28,
                                  bagging_fraction=0.8, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.05, 
                                  max_depth=6, objective='fair')
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['fair'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1+1e6
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.00504397

[1]	valid_0's fair: 0.0276439
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's fair: 0.0273023
[3]	valid_0's fair: 0.0269648
[4]	valid_0's fair: 0.0266362
[5]	valid_0's fair: 0.0262995
[6]	valid_0's fair: 0.0259747
[7]	valid_0's fair: 0.0256685
[8]	valid_0's fair: 0.0253607
[9]	valid_0's fair: 0.0250576
[10]	valid_0's fair: 0.0247713
[11]	valid_0's fair: 0.0245108
[12]	valid_0's fair: 0.0242319
[13]	valid_0's fair: 0.0239557
[14]	valid_0's fair: 0.0236973
[15]	valid_0's fair: 0.0234371
[16]	valid_0's fair: 0.0231791
[17]	valid_0's fair: 0.0229431
[18]	valid_0's fair: 0.0226862
[19]	valid_0's fair: 0.02246
[20]	valid_0's fair: 0.0222308
[21]	valid_0's fair: 0.0219981
[22]	valid_0's fair: 0.0217738
[23]	valid_0's fair: 0.0215453
[24]	valid_0's fair: 0.0213317
[25]	valid_0's fair: 0.0211197
[26]	valid_0's fair: 0.020908
[27]	valid_0's fair: 0.0206958
[28]	valid_0's fair: 0.0204896
[29]	valid_0's fair: 0.0202983
[30]	valid_0's fair: 0.0201052
[31]	valid_0's fair

[293]	valid_0's fair: 0.00812939
[294]	valid_0's fair: 0.008122
[295]	valid_0's fair: 0.00811344
[296]	valid_0's fair: 0.00810738
[297]	valid_0's fair: 0.00809874
[298]	valid_0's fair: 0.00809132
[299]	valid_0's fair: 0.00808297
[300]	valid_0's fair: 0.0080734
[301]	valid_0's fair: 0.0080663
[302]	valid_0's fair: 0.00805881
[303]	valid_0's fair: 0.00805244
[304]	valid_0's fair: 0.00804234
[305]	valid_0's fair: 0.00803476
[306]	valid_0's fair: 0.00802856
[307]	valid_0's fair: 0.00802189
[308]	valid_0's fair: 0.00801275
[309]	valid_0's fair: 0.0080051
[310]	valid_0's fair: 0.00799889
[311]	valid_0's fair: 0.00799169
[312]	valid_0's fair: 0.00798152
[313]	valid_0's fair: 0.00797445
[314]	valid_0's fair: 0.0079668
[315]	valid_0's fair: 0.00795881
[316]	valid_0's fair: 0.00795116
[317]	valid_0's fair: 0.00794639
[318]	valid_0's fair: 0.00794033
[319]	valid_0's fair: 0.00793556
[320]	valid_0's fair: 0.00792663
[321]	valid_0's fair: 0.00791946
[322]	valid_0's fair: 0.00791281
[323]	valid_0's 

[614]	valid_0's fair: 0.00688409
[615]	valid_0's fair: 0.00688206
[616]	valid_0's fair: 0.00688081
[617]	valid_0's fair: 0.00687799
[618]	valid_0's fair: 0.00687636
[619]	valid_0's fair: 0.00687302
[620]	valid_0's fair: 0.00687156
[621]	valid_0's fair: 0.00686868
[622]	valid_0's fair: 0.00686758
[623]	valid_0's fair: 0.00686599
[624]	valid_0's fair: 0.0068645
[625]	valid_0's fair: 0.00686213
[626]	valid_0's fair: 0.00686042
[627]	valid_0's fair: 0.0068583
[628]	valid_0's fair: 0.00685598
[629]	valid_0's fair: 0.00685413
[630]	valid_0's fair: 0.00685094
[631]	valid_0's fair: 0.00684914
[632]	valid_0's fair: 0.00684645
[633]	valid_0's fair: 0.00684553
[634]	valid_0's fair: 0.0068443
[635]	valid_0's fair: 0.0068429
[636]	valid_0's fair: 0.00684092
[637]	valid_0's fair: 0.00683926
[638]	valid_0's fair: 0.00683723
[639]	valid_0's fair: 0.00683467
[640]	valid_0's fair: 0.00683268
[641]	valid_0's fair: 0.00683048
[642]	valid_0's fair: 0.00682882
[643]	valid_0's fair: 0.00682732
[644]	valid_0'

[936]	valid_0's fair: 0.00649124
[937]	valid_0's fair: 0.00649145
[938]	valid_0's fair: 0.00649133
[939]	valid_0's fair: 0.0064897
[940]	valid_0's fair: 0.00648875
[941]	valid_0's fair: 0.00648792
[942]	valid_0's fair: 0.00648774
[943]	valid_0's fair: 0.00648637
[944]	valid_0's fair: 0.00648635
[945]	valid_0's fair: 0.00648596
[946]	valid_0's fair: 0.0064851
[947]	valid_0's fair: 0.00648447
[948]	valid_0's fair: 0.00648375
[949]	valid_0's fair: 0.00648312
[950]	valid_0's fair: 0.00648244
[951]	valid_0's fair: 0.00648179
[952]	valid_0's fair: 0.00648066
[953]	valid_0's fair: 0.00648014
[954]	valid_0's fair: 0.00647938
[955]	valid_0's fair: 0.0064785
[956]	valid_0's fair: 0.00647758
[957]	valid_0's fair: 0.00647609
[958]	valid_0's fair: 0.00647461
[959]	valid_0's fair: 0.00647417
[960]	valid_0's fair: 0.00647327
[961]	valid_0's fair: 0.00647243
[962]	valid_0's fair: 0.00647157
[963]	valid_0's fair: 0.00647109
[964]	valid_0's fair: 0.00647042
[965]	valid_0's fair: 0.00647027
[966]	valid_0

[1261]	valid_0's fair: 0.00633123
[1262]	valid_0's fair: 0.00633052
[1263]	valid_0's fair: 0.00633056
[1264]	valid_0's fair: 0.00633085
[1265]	valid_0's fair: 0.00633054
[1266]	valid_0's fair: 0.00633024
[1267]	valid_0's fair: 0.00633004
[1268]	valid_0's fair: 0.00632913
[1269]	valid_0's fair: 0.00632802
[1270]	valid_0's fair: 0.00632778
[1271]	valid_0's fair: 0.00632741
[1272]	valid_0's fair: 0.00632748
[1273]	valid_0's fair: 0.00632712
[1274]	valid_0's fair: 0.00632699
[1275]	valid_0's fair: 0.00632719
[1276]	valid_0's fair: 0.00632702
[1277]	valid_0's fair: 0.00632633
[1278]	valid_0's fair: 0.0063257
[1279]	valid_0's fair: 0.00632541
[1280]	valid_0's fair: 0.00632536
[1281]	valid_0's fair: 0.00632497
[1282]	valid_0's fair: 0.00632472
[1283]	valid_0's fair: 0.00632471
[1284]	valid_0's fair: 0.00632445
[1285]	valid_0's fair: 0.00632398
[1286]	valid_0's fair: 0.00632389
[1287]	valid_0's fair: 0.00632412
[1288]	valid_0's fair: 0.00632388
[1289]	valid_0's fair: 0.00632388
[1290]	valid_0'

...

[2521]	valid_0's fair: 0.00530016
[2522]	valid_0's fair: 0.00529964
[2523]	valid_0's fair: 0.00530006
[2524]	valid_0's fair: 0.00529994
[2525]	valid_0's fair: 0.00529974
[2526]	valid_0's fair: 0.00529977
[2527]	valid_0's fair: 0.00529971
[2528]	valid_0's fair: 0.00529953
[2529]	valid_0's fair: 0.00529921
[2530]	valid_0's fair: 0.00529958
[2531]	valid_0's fair: 0.00529977
[2532]	valid_0's fair: 0.00529937
[2533]	valid_0's fair: 0.00529923
[2534]	valid_0's fair: 0.00529934
[2535]	valid_0's fair: 0.00529926
[2536]	valid_0's fair: 0.00529873
[2537]	valid_0's fair: 0.00529861
[2538]	valid_0's fair: 0.00529878
[2539]	valid_0's fair: 0.00529944
[2540]	valid_0's fair: 0.00529941
[2541]	valid_0's fair: 0.0052994
[2542]	valid_0's fair: 0.00529899
[2543]	valid_0's fair: 0.00529927
[2544]	valid_0's fair: 0.00529909
[2545]	valid_0's fair: 0.00529914
[2546]	valid_0's fair: 0.00529898
[2547]	valid_0's fair: 0.00529906
[2548]	valid_0's fair: 0.00529915
[2549]	valid_0's fair: 0.00529866
[2550]	valid_0'

[2763]	valid_0's fair: 0.00529127
[2764]	valid_0's fair: 0.00529104
[2765]	valid_0's fair: 0.00529123
[2766]	valid_0's fair: 0.00529161
[2767]	valid_0's fair: 0.00529141
[2768]	valid_0's fair: 0.00529166
[2769]	valid_0's fair: 0.00529181
[2770]	valid_0's fair: 0.00529149
[2771]	valid_0's fair: 0.0052914
[2772]	valid_0's fair: 0.00529107
[2773]	valid_0's fair: 0.00529148
[2774]	valid_0's fair: 0.00529119
[2775]	valid_0's fair: 0.00529186
[2776]	valid_0's fair: 0.00529179
[2777]	valid_0's fair: 0.0052915
[2778]	valid_0's fair: 0.00529165
[2779]	valid_0's fair: 0.00529165
[2780]	valid_0's fair: 0.00529166
[2781]	valid_0's fair: 0.00529175
[2782]	valid_0's fair: 0.00529226
[2783]	valid_0's fair: 0.00529226
[2784]	valid_0's fair: 0.00529231
[2785]	valid_0's fair: 0.00529257
[2786]	valid_0's fair: 0.00529247
[2787]	valid_0's fair: 0.00529263
[2788]	valid_0's fair: 0.00529227
[2789]	valid_0's fair: 0.00529187
[2790]	valid_0's fair: 0.00529141
[2791]	valid_0's fair: 0.00529127
[2792]	valid_0's

[3083]	valid_0's fair: 0.00528107
[3084]	valid_0's fair: 0.00528117
[3085]	valid_0's fair: 0.00528135
[3086]	valid_0's fair: 0.00528117
[3087]	valid_0's fair: 0.005281
[3088]	valid_0's fair: 0.00528083
[3089]	valid_0's fair: 0.00528099
[3090]	valid_0's fair: 0.00528087
[3091]	valid_0's fair: 0.0052812
[3092]	valid_0's fair: 0.00528128
[3093]	valid_0's fair: 0.00528167
[3094]	valid_0's fair: 0.00528158
[3095]	valid_0's fair: 0.00528153
[3096]	valid_0's fair: 0.00528142
[3097]	valid_0's fair: 0.00528091
[3098]	valid_0's fair: 0.00528051
[3099]	valid_0's fair: 0.00528072
[3100]	valid_0's fair: 0.00528062
[3101]	valid_0's fair: 0.00528058
[3102]	valid_0's fair: 0.00528049
[3103]	valid_0's fair: 0.00528048
[3104]	valid_0's fair: 0.00528062
[3105]	valid_0's fair: 0.00528077
[3106]	valid_0's fair: 0.00528069
[3107]	valid_0's fair: 0.00528107
[3108]	valid_0's fair: 0.00528097
[3109]	valid_0's fair: 0.00528083
[3110]	valid_0's fair: 0.00528094
[3111]	valid_0's fair: 0.00528113
[3112]	valid_0's 

[3328]	valid_0's fair: 0.00528496
[3329]	valid_0's fair: 0.00528469
[3330]	valid_0's fair: 0.00528493
[3331]	valid_0's fair: 0.00528487
[3332]	valid_0's fair: 0.00528486
[3333]	valid_0's fair: 0.00528503
[3334]	valid_0's fair: 0.00528529
[3335]	valid_0's fair: 0.00528519
[3336]	valid_0's fair: 0.00528497
[3337]	valid_0's fair: 0.0052847
[3338]	valid_0's fair: 0.00528512
[3339]	valid_0's fair: 0.00528532
[3340]	valid_0's fair: 0.0052857
[3341]	valid_0's fair: 0.00528562
[3342]	valid_0's fair: 0.00528566
[3343]	valid_0's fair: 0.00528576
[3344]	valid_0's fair: 0.0052855
[3345]	valid_0's fair: 0.00528561
[3346]	valid_0's fair: 0.00528561
[3347]	valid_0's fair: 0.00528563
[3348]	valid_0's fair: 0.00528548
[3349]	valid_0's fair: 0.00528572
[3350]	valid_0's fair: 0.00528553
[3351]	valid_0's fair: 0.00528568
[3352]	valid_0's fair: 0.00528539
[3353]	valid_0's fair: 0.00528494
[3354]	valid_0's fair: 0.00528495
[3355]	valid_0's fair: 0.00528471
[3356]	valid_0's fair: 0.00528443
[3357]	valid_0's 

5-fold cv mean l2 0.00504397
